# Using label-studio for rapid annotation of geospatial data for airborne environmental monitoring

Environmental remote sensing using airborne image capture allows unprecedented insight into the distribution and abundance of organisms on earth. To detect and classify objects in airborne imagery, machine learning tools need to label the location and classes of objects. Here at the Weecology Lab at the University of Florida, we use label-studio to annotate airborne imagery for a variety of geospatial tasks, including tree detection in very high resolution orthophotos, and bird classification in UAV orthomosaics. Label-studio’s web interface has been key in creating labels for our machine learning workflow and keep our distributed team on the same page. Machine learning models for airborne object detection and classification require thousands of labels to create accurate detection models. In this post, I provide some key steps and uses for geospatial data labeling in label studio.

1. Converting large geospatial tiles into smaller crops in a format compatible with label-studio?
2. Creating a bounding box annotation project using the label-studio python-sdk
3. Placing pre-annotated images into label-studio using existing foundation models

The data comes from our work surveying bird colonies to support everglades restoration efforts using Unpiloted Aerial Vehivles (UAV). Let’s look at an orthomosaic image of one of those colonies. We also make use of our [DeepForest python package](), a tool for airborne model training focused on ecological monitoring. 

![Figure 1](public/Figure1.jpg)

## Preparing geospatial data for labeling

### Tiling orthomosaics into manageable pieces

This orthomosaic has been georectified, meaning that the pixels in the imagery correspond to locations on the earth’s surface. While this is crucial for matching images among surveys, or associating images with other data products, it presents a couple challenges for standard annotation workflows. Geospatial data products are often very large, making them difficult to render and awkward to work with. Tiling the image into smaller pieces can be quite useful. Using our DeepForest python package, we have a couple helpful utility functions. These are mostly wrappers from common python packages for geospatial data, like Rasterio and Geopandas, which deserve a lot of credit in helping make geospatial data easy to use in python. 

In [3]:
# pip install deepforest
from deepforest import preprocess

# see README.md for downloading data
image_path = "Vulture_crop.tif"
crops = preprocess.split_raster(
    path_to_raster=image_path,
    patch_size=1500
)
print(crops[0])

TypeError: split_raster() missing 1 required positional argument: 'annotations_file'

Let's use the great label studio python SDK to create a new project. For those already using label-studio check out settings -> labeling-interface->code, to get a plain text representation of the labeling environment. This is super useful for making duplicates of existing projects and tweaking the results in the browser. For the purposes of this blog post, let's just launch on our local machine. For your API key, set the env variable, "LABEL_STUDIO_API_KEY" in a .env file. 

In [6]:

# Import the SDK and the client module
from label_studio_sdk import Client
import os

import os

for line in open(".env"):
    key, value = line.strip().split("=")
    os.environ[key] = value

# Define the URL where Label Studio is accessible and the API key for your user account
LABEL_STUDIO_URL = 'http://localhost:8080'
API_KEY = os.environ['LABEL_STUDIO_API_KEY']

# Connect to the Label Studio API and check the connection
ls = Client(url=LABEL_STUDIO_URL, api_key=API_KEY)
ls.check_connection()

project = ls.start_project(
    title='LabelStudio SDK Example',
    label_config='''
    <View>
    <Image name="image" value="$image" zoom="true" zoomControl="true" rotateControl="true"/>
    <RectangleLabels name="label" toName="image">        
    <Label value="Great Egret" background="#FFA39E"/><Label value="Great Blue Heron" background="#D4380D"/><Label value="Wood Stork" background="#FFC069"/><Label value="Snowy Egret" background="#AD8B00"/><Label value="Anhinga" background="#D3F261"/><Label value="Unidentified White" background="#389E0D"/><Label value="White Ibis" background="#5CDBD3"/><Label value="Nest" background="#FFA39E"/><Label value="Help me!" background="#D4380D"/><Label value="Eggs" background="#FFA39E"/><Label value="Roseate Spoonbill" background="#FFA39E"/></RectangleLabels>
    </View>
    '''
)

# Add the images to the project
for crop in crops:
    project.add_file(crop)

NameError: name 'crops' is not defined

### Converting existing annotations from geospatial to image coordinates

What if we already have geospatial annotations? Very often, members of team will use common geospatial tools, such as ArcGIS or QGIS to draw annotations on to data products. One potential hurdle is transforming these geospatial coordinates into formats more commonly used by machine learning models. For example, to convert our existing annotations into the pixel coordinate plane, we want the 0,0 origin to be the top left corner, and the coordinates to be the row, column location. By providing the annotations and the corresponding RGB image, we can translate the geospatial to be expressed as pixels (row, columns). Note this an approximation that doesn't account for the curvature of the earth and is only useful on small spatial scales.

In [12]:
import geopandas as gpd
import rasterio as rio
existing_annotation_shp = "/Users/benweinstein/Dropbox/Weecology/everglades_species/labelstudio/existing_annotation.shp"
image_path = "/Users/benweinstein/Dropbox/Weecology/everglades_species/Raw_versus_Ortho/Vulture_03_28_2022.tif"
gdf = gpd.read_file(existing_annotation_shp)
with rio.open(image_path) as src:
    left, bottom, right, top = src.bounds
    image_resolution = src.res[0]
left, bottom, right, top = src.bounds
image_resolution = src.res[0]

gdf.geometry = gdf.geometry.translate(xoff=-left, yoff=-top)
gdf.geometry = gdf.geometry.scale(xfact=1/image_resolution, yfact=-1/image_resolution, origin=(0,0))
gdf.crs = None

,id,geometry
0,None,"POLYGON ((6876.169 28793.212, 6876.169 28729.4..."


## Foundation models for pre-annotation makes labeling faster.

Now that we are ready to annotate, we find that fully annotating even the tiled data still takes a long time, since we spend alot of time zooming and panning to find all the birds. This is where the recent growth in open-source machine learning models makes all the difference. By leveraging existing broad classifiers we can pre-annotate our label studio workflow with 'Bird' label. See [DeepForest docs]() for more on these foundation models.

In [ ]:
#Code Block 3 Simple upload of DeepForest annotations. 

After humans have reviewed and corrected these annotations, we can download them to train machine learning models using the web interface or label-studio’s python API.

## Geospatial predictions from image-coordinates

Once we’ve made out predictions, we need to convert them back into geospatial coordinates to match our original data. This means going from boxes with image-based coordinates relative to the 0,0 origin to boxes with geospatial coordinates relative to the native map projection.  

In [ ]:
with rasterio.open(rgb_path) as dataset:
    bounds = dataset.bounds
    left, bottom, right, top = bounds
    pixelSizeX, pixelSizeY = dataset.res
    crs = dataset.crs

xmin_coords, ymin_coords = rasterio.transform.xy(transform=dataset.transform,
                                            rows=coordinates.miny,
                                xs            cols=coordinates.minx,
                                            offset='center')

xmax_coords, ymax_coords = rasterio.transform.xy(transform=dataset.transform,
                                            rows=coordinates.maxy,
                                            cols=coordinates.maxx,
                                            offset='center')

![final raster with geospatial predictions]()

## How to make this faster and more accurate?

Label studio’s open-source API opens the possibility for community driven development. We need your help! There are many ways to make annotation faster, which means more data, better models, and better information to make conservation decisions. We would welcome developers looking to make a difference to help make label-studio more ready for geospatial data.

1.	Allow overlays of multiple products. Often, we have several data types, such as RGB data, airborne multi-spectral data and LiDAR data that we want to look at in unison to help guide annotation. While this is possible in tools like QGIS, it is not natively doable in label-studio. 
2.	Intelligent zooming to annotations. Double clicking on the UI should zoom to a particular annotation, reducing costly scrolling time among objects. 
3. Improved backend integrations! The future of data labeling is interactive collaboration with foundation machine learning models. In the example above we preprocessed an image with a model and served the result to a person. An even better strategy is to take input from a person, then pass it to a model, and return it. This is starting to make its way into label-studio, with Meta's Segment-Anything model. A flavor of this model, called SAM-geo has been trained for remote sensing, and can be directly integrated into the server backend to provide segmentations based on prompts. These predicted annotations can then be quickly cleaned up by a human at a fraction of the total time needed to create polygon annotations. 


# Acknowlegements

The everglades bird survey gratefully acknowledges support from the South Florida Water Management District, US Army Corps of Engineers and University of Florida. Lindsey Gardner flew the UAV mission and Henry Senyondo processed the orthomosaic data. The DeepForest python package was supported by the National Science Foundation. The Weecology lab has no conflicts of interest or funding  